In [1]:
# Importing libraries
import numpy as np
from scipy.signal import stft, istft

In [13]:
# This will do Wiener Filtering on a signal. Parameters: noisy signal and sampling frequency
# We will estimate the Power of the noise from the first few samples and Calculate the Wiener gain.
# Thanks to Wiener gain Formula, now we have control over frequencies, instead of blind subtraction.
# Certain frequencies get attenuated less and more, depending on the Power Spectral Density ratio.
# Again, We reconstruct the signal back to the time domain using the original phase.

def wiener_filtering(signal_noisy, fs, alpha):
    
    f, t, complex_spectr = stft(signal_noisy, fs, nperseg = 1024, nfft = 1024, noverlap = 512, window = 'hann') # complex_spectr will also contain phase

    # Averaging across the first few samples as the voice is silent there, but noise exists and calculating its power. 
    noise_psd = np.mean(np.abs(complex_spectr[:,:50])**2, axis = 1, keepdims = True) 

    # Power of noisy signal
    signal_psd = np.abs(complex_spectr)**2

    H = signal_psd / (signal_psd + alpha * noise_psd) # Wiener gain
    H_scaled = np.zeros_like(H) 
    # Done to scale Wiener gain values (Values close to 0 get pushed harder to 0 and values close to 1 get pushed to 1)
    H_scaled[H < 0.6] = H[H < 0.6]**4
   # H_scaled[(H >= 0.6) & (H < 0.7)] = 1 - (1 - H[(H >= 0.6) & (H < 0.7)])**4
    H_scaled[H >= 0.6] = 1 - (1 - H[H >= 0.6])**5
    
    X = complex_spectr * H_scaled

    t, signal_denoised = istft(X, fs, nperseg = 1024, nfft = 1024, noverlap = 512, window = 'hann')
    signal_denoised = np.clip(signal_denoised, -1, 1)

    return signal_denoised